In [1]:
import zipfile
import os
import re
from re import finditer
import os
import pandas as pd

In [28]:
os.chdir('c://users//f33718//desktop//docx specs')

In [29]:
# GET SAFETY CLEARANCE CODE:
def getSafetyClearance(xml):
    safety = []
    for match in finditer('C[0-9]{4}-[0-9]{4}', xml):
        safety.append(match.group())
    for match in finditer('IPSCS-RM-[0-9]{4}-[0-9]{4}', xml):
        safety.append(match.group())
    for match in finditer('1[0-9]{11}', xml):
        safety.append(match.group())
    new_spec['safety'].append(safety)

In [30]:
# GET SAP PRODUCT CODE(S):
def getSapProductCode(xml):
    sap_codes = []
    for match in finditer('>4[0-9]{7}<', xml):
        sap_codes.append(match.group())
    for i in range(len(sap_codes)):
        sap_codes[i] = sap_codes[i].replace('>','')
        sap_codes[i] = sap_codes[i].replace('<','')
    new_spec['sap_codes'].append(sap_codes)

In [31]:
# GET VENDOR / KCC EMAIL(S):
def getEmails(xml):
    emails = []
    for match in finditer('[a-zA-Z0-9_.]+@[a-zA-Z0-9_.]+\.com', xml):
        emails.append(match.group())
    new_spec['emails'].append(emails)

In [32]:
# GET SPEC NUMBER
def getSpecNumber(header):
    spec_num = []
    for match in finditer('SPEC-[0-9]{5}', header):
        spec_num.append(match.group())
    new_spec['spec_number'].append(spec_num)

In [33]:
# GET REVISION NUMBER
def getRevision(header):
    number = []
    for match in finditer('>[0-9]<', header):
        number = match.group()
        number = number.replace('>', '')
        number = number.replace('<', '')
    new_spec['revision'].append(number)

In [34]:
def grabData(xml,header):
    getSafetyClearance(xml)
    getSapProductCode(xml)
    getEmails(xml)
    getRevision(header)
    getSpecNumber(header)

In [35]:
new_spec = {
    'spec_number':[],
    'safety':[],
    'emails':[],
    'sap_codes':[],
    'revision':[]
}

In [36]:


for file in os.listdir():
    document = zipfile.ZipFile(file)
    spec = document.read('word/document.xml').decode()

    if 'word/header1.xml' in document.namelist():
        header = document.read('word/header1.xml').decode()
        grabData(spec, header)
    else:
        header = ' - '
        grabData(spec, header)


In [37]:
os.chdir('c://users//f33718//desktop')
word_specs = pd.DataFrame.from_dict(new_spec)
word_specs.to_excel('SPEC Sheet Data (Word).xlsx')